In [68]:
import os
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

import warnings
warnings.filterwarnings('ignore')

In [69]:
aws_psql_conn = psycopg2.connect(
    database=os.getenv("AWS_PSQL_DB"),
    user=os.getenv("AWS_PSQL_USER"),
    password=os.getenv("AWS_PSQL_PASSWORD"),
    host=os.getenv("AWS_PSQL_HOST"),
    port=os.getenv("AWS_PSQL_PORT"),
)

cursor = aws_psql_conn.cursor()

In [70]:
sql = f"select * from games"

cursor.execute(sql)

rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]

In [71]:
df = pd.DataFrame(rows, columns=column_names)

print(df.head)

<bound method NDFrame.head of       game_id  home_team_id       home_team_name  away_team_id  \
0      718698           158    Milwaukee Brewers           121   
1      718781           147     New York Yankees           137   
2      718782           111       Boston Red Sox           110   
3      718777           112         Chicago Cubs           158   
4      718776           139       Tampa Bay Rays           116   
...       ...           ...                  ...           ...   
1199   717584           112         Chicago Cubs           143   
1200   717433           147     New York Yankees           112   
1201   717581           121        New York Mets           158   
1202   717582           138  St. Louis Cardinals           117   
1203   717580           140        Texas Rangers           116   

             away_team_name      home_pitcher  home_pitcher_id  \
0             New York Mets     Corbin Burnes         669203.0   
1      San Francisco Giants       Gerrit Cole

In [72]:
columns_to_remove = [
    "home_team_name",
    "away_team_name",
    "home_pitcher",
    "home_pitcher_id",
    "away_pitcher",
    "away_pitcher_id",
]

df = df.drop(columns=columns_to_remove)

print(df.head)

<bound method NDFrame.head of       game_id  home_team_id  away_team_id  home_pitcher_era  \
0      718698           158           121              7.20   
1      718781           147           137               NaN   
2      718782           111           110               NaN   
3      718777           112           158               NaN   
4      718776           139           116               NaN   
...       ...           ...           ...               ...   
1199   717584           112           143              3.38   
1200   717433           147           112              4.52   
1201   717581           121           158              3.52   
1202   717582           138           117              4.23   
1203   717580           140           116              2.76   

      home_pitcher_win_percentage  home_pitcher_wins  home_pitcher_losses  \
0                           0.000                0.0                  1.0   
1                             NaN                NaN       

In [73]:
df = df.dropna()

print(df.head)

<bound method NDFrame.head of       game_id  home_team_id  away_team_id  home_pitcher_era  \
0      718698           158           121              7.20   
72     718703           119           115              3.00   
73     718704           120           139              6.00   
74     718701           146           142              0.00   
77     717821           116           109              5.23   
...       ...           ...           ...               ...   
1198   717888           119           147              1.64   
1199   717584           112           143              3.38   
1201   717581           121           158              3.52   
1202   717582           138           117              4.23   
1203   717580           140           116              2.76   

      home_pitcher_win_percentage  home_pitcher_wins  home_pitcher_losses  \
0                           0.000                0.0                  1.0   
72                          1.000                1.0       

In [74]:
df.columns

Index(['game_id', 'home_team_id', 'away_team_id', 'home_pitcher_era',
       'home_pitcher_win_percentage', 'home_pitcher_wins',
       'home_pitcher_losses', 'home_pitcher_innings_pitched',
       'away_pitcher_era', 'away_pitcher_win_percentage', 'away_pitcher_wins',
       'away_pitcher_losses', 'away_pitcher_innings_pitched', 'winning_team',
       'home_pitcher_k_nine', 'home_pitcher_bb_nine', 'home_pitcher_k_bb_diff',
       'home_pitcher_whip', 'home_pitcher_babip', 'away_pitcher_k_nine',
       'away_pitcher_bb_nine', 'away_pitcher_k_bb_diff', 'away_pitcher_whip',
       'away_pitcher_babip', 'predicted_winner'],
      dtype='object')

In [75]:
X = []
Y = []

for index, row in df.iterrows():
    pitcher_innings_pitched_comp = row['away_pitcher_innings_pitched'] - row['home_pitcher_innings_pitched']
    pitcher_k_nine_comp = row['away_pitcher_k_nine'] - row['home_pitcher_k_nine']
    pitcher_bb_nine_comp = row['away_pitcher_bb_nine'] - row['home_pitcher_bb_nine']
    pitcher_k_bb_diff_comp = row['away_pitcher_k_bb_diff'] - row['home_pitcher_k_bb_diff']
    pitcher_whip_comp = row['away_pitcher_whip'] - row['home_pitcher_whip']
    pitcher_babip_comp = row['away_pitcher_babip'] - row['home_pitcher_babip']

    comparison = [pitcher_innings_pitched_comp, pitcher_k_nine_comp, pitcher_bb_nine_comp, pitcher_k_bb_diff_comp, pitcher_whip_comp, pitcher_babip_comp]

    X.append(comparison)
    Y.append(1 if row['winning_team'] == row['home_team_id'] else 0)


In [76]:
# Convert the lists to NumPy arrays for compatibility with scikit-learn
X = pd.np.array(X)
Y = pd.np.array(Y)
X = X.reshape(X.shape[0], -1)

In [77]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Print the shape of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (661, 6)
X_test shape: (166, 6)
Y_train shape: (661,)
Y_test shape: (166,)


In [78]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [79]:
model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

In [80]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [81]:
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)


Epoch 1/10


21/21 [==============================] - 0s 557us/step - loss: 0.8459 - accuracy: 0.4750
Epoch 2/10
21/21 [==============================] - 0s 593us/step - loss: 0.8093 - accuracy: 0.4705
Epoch 3/10
21/21 [==============================] - 0s 458us/step - loss: 0.7808 - accuracy: 0.4720
Epoch 4/10
21/21 [==============================] - 0s 522us/step - loss: 0.7589 - accuracy: 0.4690
Epoch 5/10
21/21 [==============================] - 0s 476us/step - loss: 0.7435 - accuracy: 0.4766
Epoch 6/10
21/21 [==============================] - 0s 436us/step - loss: 0.7302 - accuracy: 0.4750
Epoch 7/10
21/21 [==============================] - 0s 589us/step - loss: 0.7209 - accuracy: 0.4856
Epoch 8/10
21/21 [==============================] - 0s 448us/step - loss: 0.7140 - accuracy: 0.4947
Epoch 9/10
21/21 [==============================] - 0s 433us/step - loss: 0.7089 - accuracy: 0.5023
Epoch 10/10
21/21 [==============================] - 0s 456us/step - loss: 0.7042 - accuracy: 0.5144


In [82]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


6/6 [==============================] - 0s 722us/step - loss: 0.7049 - accuracy: 0.5000
Test Loss: 0.7049
Test Accuracy: 0.5000
